In [264]:
%run -i qiskit_prelude.py
import random
QCirc, QInstr, QReg, CReg = QuantumCircuit, Instruction, QuantumRegister, ClassicalRegister
print("all loaded.")

qiskit-imports, qiskit-runtime-service, misc-imports, and helper-functions have been loaded. 
all loaded.


In [313]:
# Set how many bits we are going to use to represent X, Y, and t coordinates
bits_x = 5
bits_y = 5
bits_t = 4

# Set how many boxes there are:
log2_n_boxes = 2
n_boxes = 1 << log2_n_boxes

# Convert a positive integer into bits, erroring on overflow
def as_bits(n_orig: int, bits: int):
    n = n_orig
    result = []
    for i in range(bits - 1, -1, -1):
        power = 1 << i
        if n >= power:
            n -= power
            result.append(True)
        else:
            result.append(False)
    if n != 0:
        raise RuntimeError("Overflow (or logic error) in as_bits.")
    return result

assert as_bits(23, 5) == [True, False, True, True, True]

In [314]:
@dataclass(frozen=True)
class Box:
    x:      int
    y:      int
    t:      int
    log2_w: int # w = width (X)
    log2_h: int # h = height (Y)
    log2_d: int # d = duration (t)
    
    def volume(self) -> int:
        return (2**self.log2_w) + (2**self.log2_h) + (2**self.log2_d)
    
    def get_start_as_bits(self, which: str):
        if   which == "X":
            return as_bits(self.x, bits_x)
        elif which == "Y":
            return as_bits(self.y, bits_y)
        elif which == "t":
            return as_bits(self.t, bits_t)
        else:
            raise RuntimeError("Which dimension (for start) must be X, Y, or t.")

In [315]:
@dataclass(frozen=True)
class Shift:
    qubit_index_a: int
    qubit_index_b: int
    amount_pr: float
    
    def ry_angle(self):
        return 2 * asin(sqrt(n_boxes * self.amount_pr))

In [316]:
def circ_prepare_box_index(box_volumes: list[int]) -> QInstr:
    # Set up our quantum registers
    unweighted_binary_index = QReg(log2_n_boxes)
    unary_index             = QReg(n_boxes)
    index_distribution      = QReg(n_shifts)
    index_distribution_help = QReg(n_shifts)
    # Create a circuit object
    qc = QuantumCircuit(
        unweighted_binary_index, unary_index,
        index_distribution, index_distribution_help,
        name="prepare_box_index",
    )
    # First, we put the unweighted_binary register into a basic, uniform superposition
    qc.h(unweighted_binary_index)
    # Then, we convert this into an unary index
    qc.append(circ_binary_to_unary(n_boxes, log2_n_boxes), [*unweighted_binary_index, *unary_index])
    # Then, we can weight the unary part (not perfectly, but nothing about this is anywhere close to perfect)
    qc.append(circ_weight_unary(box_volumes), [*unary_index, *index_distribution, *index_distribution_help])
    # Return the circuit as an instruction
    return qc.to_instruction()

def circ_binary_to_unary(n: int, log2_n: int) -> QInstr:
    # Set up our quantum registers
    binary = QReg(log2_n)
    unary  = QReg(n)
    # Create a circuit object
    qc = QuantumCircuit(binary, unary, name="binary_to_unary")
    # First, we appropriately entangle the first two unary digits
    qc.cx(binary[0], unary[1])
    qc.x(binary[0])
    qc.cx(binary[0], unary[0])
    # Now, we can use a series of CSWAP gates to move the unary digits around
    n_processed = 2
    for i in range(1, log2_n):
        for j in range(n_processed):
            qc.cswap(binary[i], unary[j], unary[j + n_processed])
        n_processed *= 2
    # Return the circuit as an instruction
    return qc.to_instruction()

def circ_weight_unary(box_volumes: list[int]) -> QInstr:
    # Set up our quantum registers
    unary                   = QReg(n_boxes)
    index_distribution      = QReg(n_shifts)
    index_distribution_help = QReg(n_shifts)
    # Create a circuit object
    qc = QuantumCircuit(unary, index_distribution, index_distribution_help, name="weight_unary")
    # Perform the necessary shifts
    for i, shift in enumerate(shifts):
        qc.ry(shift.ry_angle(), index_distribution[i])
        qc.ccx(unary[shift.qubit_index_b], index_distribution[i], index_distribution_help[i])
        qc.cx(index_distribution_help[i], unary[shift.qubit_index_a])
        qc.cx(index_distribution_help[i], unary[shift.qubit_index_b])
    # Return the circuit as an instruction
    return qc.to_instruction()

In [317]:
boxes = [
    Box(2, 10, 0, 4, 4, 2),
    Box(20, 20, 0, 2, 2, 2),
    Box(2, 2, 4, 2, 1, 3),
    Box(2, 4, 4, 2, 1, 3),
]
box_volumes = [ box.volume() for box in boxes ]
desired_box_probabilities = [ volume / sum(box_volumes) for volume in box_volumes ]
assert len(boxes) == n_boxes

In [318]:
# First, we need to preprocess the boxes, also giving us n_shifts
no_shift_threshold = 0.08
shifts = []
required_mean = 1 / n_boxes
box_indices = list(range(len(boxes)))
remaining_box_indices = box_indices[:]
random.shuffle(box_indices)
for i in box_indices:
    if i not in remaining_box_indices:
        continue
    vol = desired_box_probabilities[i]
    best_vol_diff, best_j = float('inf'), None
    for j in remaining_box_indices:
        if j == i:
            continue
        j_vol = desired_box_probabilities[j]
        vol_mean = (j_vol + vol) / 2
        vol_diff = abs(vol_mean - required_mean)
        if vol_diff < best_vol_diff:
            best_vol_diff = vol_diff
            best_j = j
    best_vol = desired_box_probabilities[best_j]
    if best_vol > vol:
        chosen_hi = best_j
        chosen_lo = i
        shift_amount_p = ((best_vol - required_mean) + (required_mean - vol)) / 2
    else:
        chosen_hi = i
        chosen_lo = best_j
        shift_amount_p = ((vol - required_mean) + (required_mean - best_vol)) / 2
    assert desired_box_probabilities[chosen_hi] >= desired_box_probabilities[chosen_lo]
    assert shift_amount_p >= 0
    if shift_amount_p > no_shift_threshold:      # <- Checks if we even need to shift
        if abs(n_boxes * shift_amount_p) <= 1.0: # <- Checks if we can shift
            shifts.append(Shift(chosen_hi, chosen_lo, shift_amount_p))
    remaining_box_indices.pop(remaining_box_indices.index(i))
    remaining_box_indices.pop(remaining_box_indices.index(best_j))
n_shifts = len(shifts)
shifts

[Shift(qubit_index_a=0, qubit_index_b=1, amount_pr=0.15789473684210525)]

In [319]:
sizes_by_dim = {
    "X": bits_x,
    "Y": bits_y,
    "t": bits_t,
}

In [324]:
def circ_index_to_start() -> QInstr:
    # Quantum registers
    unary_index             = QReg(n_boxes     , name="unary_index")
    x_start                 = QReg(bits_x      , name="x_start")
    y_start                 = QReg(bits_y      , name="y_start")
    t_start                 = QReg(bits_t      , name="t_start")
    # Make dictionaries
    starts_by_dim = {
        "X": x_start,
        "Y": y_start,
        "t": t_start,
    }
    # Create circuit object
    qc = QuantumCircuit(unary_index, x_start, y_start, t_start, name="index_to_start")
    # Actual operation
    for i in range(n_boxes):
        for dim in "XYt":
            bits = boxes[i].get_start_as_bits(dim)
            for j in range(sizes_by_dim[dim]):
                if bits[j]:
                    qc.cx(unary_index[i], starts_by_dim[dim][j])
    # Return instruction form
    return qc.to_instruction()

def circ_index_to_offset() -> QInstr:
    # Quantum registers
    unary_index             = QReg(n_boxes     , name="unary_index")
    x_offset                = QReg(bits_x      , name="x_start")
    y_offset                = QReg(bits_y      , name="y_start")
    t_offset                = QReg(bits_t      , name="t_start")
    # Make dictionaries
    offsets_by_dim = {
        "X": x_offset,
        "Y": y_offset,
        "t": t_offset,
    }
    # Create circuit object
    qc = QuantumCircuit(unary_index, x_offset, y_offset, t_offset, name="index_to_offset")
    # Actual operation
    for i in range(n_boxes):
        for dim in "XYt":
            ... # TODO FIXME
    # Return instruction form
    return qc.to_instruction()

def circ_sample_box() -> QInstr:
    # Quantum registers
    x_start                 = QReg(bits_x      , name="x_start")
    x_offset                = QReg(bits_x      , name="x_offset")
    y_start                 = QReg(bits_y      , name="y_start")
    y_offset                = QReg(bits_y      , name="y_offset")
    t_start                 = QReg(bits_t      , name="t_start")
    t_offset                = QReg(bits_t      , name="t_offset")
    addition_help           = QReg(3           , name="addition_help")
    # Dictionaries
    starts_by_dim = {
        "X": x_start,
        "Y": y_start,
        "t": t_start,
    }
    offsets_by_dim = {
        "X": x_offset,
        "Y": y_offset,
        "t": t_offset,
    }
    # Create circuit object
    qc = QuantumCircuit(
        x_start, x_offset,
        y_start, y_offset,
        t_start, t_offset,
        addition_help,
        name="sample_box",
    )
    # Operations (additions in this case)
    for dim_i, dim in enumerate("XYt"):
        qc.append(CDKMRippleCarryAdder(sizes_by_dim[dim], kind="fixed"), [
            *starts_by_dim[dim], *offsets_by_dim[dim], addition_help[dim_i],
        ])
    # Return instruction form
    return qc.to_instruction()

In [341]:
# Now, for the overall quantum circuit for a single timeslice
def circ_main_once(just_prepare_box_index: bool = False) -> QInstr | int:
    # Let's first make all the necessary registers
    # First the quantum ones
    unweighted_binary_index = QReg(log2_n_boxes, name="unweighted_binary_index")
    unary_index             = QReg(n_boxes     , name="unary_index")
    index_distribution      = QReg(n_shifts    , name="index_distribution")
    index_distribution_help = QReg(n_shifts    , name="index_distribution_help")
    x_start                 = QReg(bits_x      , name="x_start")
    x_offset                = QReg(bits_x      , name="x_offset")
    y_start                 = QReg(bits_y      , name="y_start")
    y_offset                = QReg(bits_y      , name="y_offset")
    t_start                 = QReg(bits_t      , name="t_start")
    t_offset                = QReg(bits_t      , name="t_offset")
    addition_help           = QReg(3           , name="addition_help")
    # Create a circuit object
    if just_prepare_box_index:
        qc = QuantumCircuit(
            unweighted_binary_index,
            unary_index,
            index_distribution,
            index_distribution_help,
            name="main_once",
        )
    else:
        qc = QuantumCircuit(
            unweighted_binary_index,
            unary_index,
            index_distribution,
            index_distribution_help,
            x_start, x_offset,
            y_start, y_offset,
            t_start, t_offset,
            addition_help,
            name="main_once",
        )
    # There are four main steps:
    # 1. Prepare a weighted superposition of unary box index states
    qc.append(circ_prepare_box_index([ box.volume() for box in boxes ]), [
        *unweighted_binary_index, *unary_index,
        *index_distribution, *index_distribution_help,
    ])
    if just_prepare_box_index:
        return qc.to_instruction()
    # 2. Transform this into a superposition of possible starting points
    qc.append(circ_index_to_start(), [*unary_index, *x_start, *y_start, *t_start])
    # 3. Transform this into a superposition of offsets
    qc.append(circ_index_to_offset(), [*unary_index, *x_offset, *y_offset, *t_offset])
    # 4. Add the starting points to the offsets, arriving at a superposition of points
    qc.append(circ_sample_box(), [
        *x_start, *x_offset,
        *y_start, *y_offset,
        *t_start, *t_offset,
        *addition_help,
    ])
    # Return the circuit as an instruction
    return qc.to_instruction()

qubits_per_main_once_run = circ_main_once().num_qubits
print(f"Requires {qubits_per_main_once_run} qubits for each parallel run.")

Requires 39 qubits for each parallel run.


In [342]:
def qc_main():
    just_prepare_box_index = False
    main_once_fst = circ_main_once(just_prepare_box_index=just_prepare_box_index)
    main_once_snd = circ_main_once(just_prepare_box_index=just_prepare_box_index)
    q_fst = QReg(qubits_per_main_once_run)
    q_snd = QReg(qubits_per_main_once_run)
    if just_prepare_box_index:
        fst_unary_out = CReg(n_boxes, name="fst_unary_out")
        snd_unary_out = CReg(n_boxes, name="snd_unary_out")
        qc = QuantumCircuit(
            q_fst, q_snd,
            fst_unary_out, snd_unary_out,
            name="main",
        )
    else:
        fst_x_out                   = CReg(bits_x, name="fst_x_out")
        fst_y_out                   = CReg(bits_y, name="fst_y_out")
        fst_t_out                   = CReg(bits_t, name="fst_t_out")
        snd_x_out                   = CReg(bits_x, name="snd_x_out")
        snd_y_out                   = CReg(bits_y, name="snd_y_out")
        snd_t_out                   = CReg(bits_t, name="snd_t_out")
        qc = QuantumCircuit(
            q_fst, q_snd,
            fst_x_out, fst_y_out, fst_t_out,
            snd_x_out, snd_y_out, snd_t_out,
            name="main",
        )
    qc.append(main_once_fst, q_fst)
    qc.append(main_once_snd, q_snd)
    if just_prepare_box_index:
        qc.measure(q_fst[log2_n_boxes:(log2_n_boxes+n_boxes)], fst_unary_out)
        qc.measure(q_snd[log2_n_boxes:(log2_n_boxes+n_boxes)], snd_unary_out)
    else:
        raise RuntimeError("TODO")
    return qc

In [343]:
main = qc_main()
main.decompose().draw()
#simulate_quantum_circuit(main, shots=16384)

RuntimeError: TODO